### Imports

In [11]:
#startup template for Jupyter Notebook
# activate tab completion so I won't shy away from descriptive vars
%config IPCompleter.use_jedi = False
%config IPCompleter.greedy = True
%matplotlib inline

In [51]:
# imports
import warnings
warnings.filterwarnings('ignore')
from flipside import Flipside
import requests
import json
import pandas as pd 
import os
from dotenv import load_dotenv
import time

In [52]:
xmode minimal

Exception reporting mode: Minimal


### Three step Process
  1. Create a query run: `createQueryRun`
  2. Poll the status of the query: `getQueryRun`
  3. Get the results: `getQueryRunResults`

'/Users/davidenoch/Documents/Projects/FLipside SDK and API'

In [86]:
def query_flipside(sql, url='https://api-v2.flipsidecrypto.xyz/json-rpc', **kwargs):
    load_dotenv()
    headers = {
  'Content-Type': 'application/json',
  'x-api-key': os.environ.get('FLIPSIDE_API_KEY')
}
    def create_query_run():
        
        params = [
    {
        "resultTTLHours": 17,
        "maxAgeMinutes": 0,
        "sql": sql,
        "dataSource": "snowflake-default",
        "dataProvider": "flipside"
    }
    ]
        if kwargs != {}:
            for key, value in kwargs:
                params[0][key] = value
    
        payload = json.dumps({
                "jsonrpc": "2.0",
                "method": "createQueryRun",
                "params": params,
                "id": 42,
                "dataSource": "snowflake-default",
                "dataProvider": "flipside"
                })
        
        #print('inside inner function')
        try:
            response = requests.request("POST", url, headers=headers, data=payload)
            if response.status_code != 200 :
                print (f"API call received status: {response.status_code}. Exiting")
                return False
                # print(f"Status code: {response.status_code}")
            # print(f"Response text: {response.text}")
        except: 
            print("error with API request in create_query_run")
            return False
        #print(response.text)
        try: 
            if response.json()['result']['queryRun']['state'] == "QUERY_STATE_READY":
                query_run_id = response.json()['result']['queryRun']['id']
                print(json.dumps(response.json()['result']['queryRequest'], indent=4))
                return query_run_id
            else:
                print("Query state is not ready")
                return False
        except MemoryError: 
            #print(f"error with api request: {response.json()['error']['message']}")
            #print("response object has no result key")
            #print(response.text)
            return False

    
    ################################################################################
    #
    # Get Query Run 
    #
    ################################################################################
    
    def get_query_run():
        load_dotenv()
        payload = json.dumps({
          "jsonrpc": "2.0",
          "method": "getQueryRun",
          "params": [
            {
              "queryRunId": query_run_id
            }
          ],
          "id": 1
        })
        done = False
        count = 0
        while not done:
            response = requests.request("POST", url, headers=headers, data=payload)
            state = response.json()['result']['queryRun']['state']
            if state == "QUERY_STATE_SUCCESS":
                print("Success.")
                return True 
            if state != "QUERY_STATE_RUNNING":
                print(f"State {state} occured")
                return False
            print(f"status is 'running' count = {count}")
            count += 1
            time.sleep(3)
            if count > 10:
                print("Taking to long. Exiting.")
            
            
            #print(json.dumps(response.json(), indent=4))
     ################################################################################
    #
    # Get Query Run Results
    #
    ################################################################################
    
    def get_query_run_results():
        payload = json.dumps({
  "jsonrpc": "2.0",
  "method": "getQueryRunResults",
  "params": [
    {
      "queryRunId": query_run_id,
      "format": "json",
      "page": {
        "number": 1,
        "size": 10
      }
    }
  ],
  "id": 1
    })
        response = requests.request("POST", url, headers=headers, data=payload)
        return response.json()
        
    ################################################################################
    #
    # Main Body of query_flipside
    #
    ################################################################################
    
    query_run_id = create_query_run()
    if not query_run_id:
        return False
    else:
        print(f"Ready to send query_id '{query_run_id}' to `get_query_run`")
        if get_query_run():
            print("Query success. Ready to get results!")
            results = get_query_run_results()
            df = pd.DataFrame(results['result']['rows'])
            return df
        
 
    

In [87]:
chain = 'ethereum'
sql = f"""
select * from {chain}.core.fact_transactions 
where block_timestamp > current_timestamp() - interval '1 days' 
limit 5
"""


df = query_flipside(sql, )
df

{
    "id": "clxju9eor2r2joi0tg6jtt89w",
    "sqlStatementId": "clxgs1ty30xm3n50tx6o25rrv",
    "userId": "clgztx7jw00qxob0sc634x5rg",
    "tags": {},
    "maxAgeMinutes": 0,
    "resultTTLHours": 17,
    "userSkipCache": true,
    "triggeredQueryRun": true,
    "queryRunId": "clxju9eo12r2hoi0t3053d7f8",
    "createdAt": "2024-06-18T03:22:28.000Z",
    "updatedAt": "2024-06-18T03:22:28.000Z"
}
Ready to send query_id 'clxju9eo12r2hoi0t3053d7f8' to `get_query_run`
status is 'running' count = 0
Success.
Query success. Ready to get results!


,block_number,block_timestamp,block_hash,tx_hash,nonce,position,origin_function_signature,from_address,to_address,value,...,tx_type,chain_id,blob_versioned_hashes,max_fee_per_blob_gas,blob_gas_used,blob_gas_price,fact_transactions_id,inserted_timestamp,modified_timestamp,__row_index
0,20114940,2024-06-18T00:00:11.000Z,0xb555d306d234d2f7e858fb5544951bce9510a24cb211...,0x1411095b3b801349ced803ad14af9b9effad26306f03...,0,231,0x,0x9f8c79e3baf81a9cf3a50b189007ad535eab6646,0x3a09c40f8f7b93c8c7e09a5422e56dfe7c7d2794,0.169922,...,0,1,None,None,None,None,0a18b6c698ad1ed6791cd131363689bf,2024-06-18T00:29:30.072Z,2024-06-18T00:29:30.072Z,0
1,20114945,2024-06-18T00:01:11.000Z,0xb4523b896ffc2aeb7cc166caded625e470353ed5620d...,0x7a7c31b7c193995d8465d2f32dc937f38303da4de6e9...,1617,19,0x791ac947,0x4c2ef512940d890dfa8c84f1a96c7f7ac2372ace,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0.000000,...,2,1,None,None,None,None,ac221045409becb8a6482ce8314c59f1,2024-06-18T00:29:30.072Z,2024-06-18T00:29:30.072Z,1
2,20114941,2024-06-18T00:00:23.000Z,0xc35e9b1c9ea3e9df6164760e111c8a89ccb3421542d3...,0x29586c4cd8743d10fabe4280c9b563a0fe53f09695fb...,54361,96,0x,0x924ccce8c07b88263a431a15c8fc4870ba81fccf,0xe61cdab7850842025f02332412f679a51fb9ec2f,0.003780,...,0,1,None,None,None,None,2f5db857f257c9dda198d65cf4fa64bb,2024-06-18T00:29:30.072Z,2024-06-18T00:29:30.072Z,2
3,20114944,2024-06-18T00:00:59.000Z,0x0677b01124a011c38dfa88cad1f4dee2524b2e7acb1c...,0xc04cd0ef4aabb7294e79a5578b9296a15be7f1df2786...,195,124,0x415565b0,0xf2b6a6d494c66969d7be3f972ef34c3db1753079,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0.000000,...,2,1,None,None,None,None,ecc2c4a643e873e5fc5d988fe3c1eb50,2024-06-18T00:29:30.072Z,2024-06-18T00:29:30.072Z,3
4,20114941,2024-06-18T00:00:23.000Z,0xc35e9b1c9ea3e9df6164760e111c8a89ccb3421542d3...,0x70c872e6fc4966879211e52052a1618813ca77cdb708...,1,183,0xa9059cbb,0x9c1113e451d52913f772a51d8405db4443ebca16,0xaea46a60368a7bd060eec7df8cba43b7ef41ad85,0.000000,...,2,1,None,None,None,None,ecd1041c8b4fe6684d6af480b9a490c0,2024-06-18T00:29:30.072Z,2024-06-18T00:29:30.072Z,4
